# GPT2 -> Generative Pretrained Transformer (2020 - 2021)

In [ ]:
%pip install transformers tiktoken

In [7]:
from dataclasses import dataclass # reduced class
import torch.nn as nn
import torch
from torch.nn import functional as F 


class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_attn = nn.Linear(config.n_embd, 3*config.n_embd) # create Q, K, V 
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)

        self.n_head = config.n_head
        self.n_embd = config.n_embd
    
    def forward(self, x):
        B, T , C = x.size() # batch_size, T .. sequence length , C = n_embd 
        qkv = self.c_attn(x)
        q,k,v = qkv.split(self.n_embd, dim = 2) 
        # n_head = nh, C = nh * hs, hs is head size (in our case 64)
        # C = 768, n_head = 12, hs = 64
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        y = F.scaled_dot_product_attention(q,k,v, is_causal=True) # causal = masked 
        # Attention(q,k,v) = softmax(q*k/sqrt(C))*V 
        y = y.transpose(1, 2).contiguous().view(B, T, C) # connect the result of heads again in to one vector
        # output projection 
        y = self.c_proj(y)
        return y


class MLP(nn.Module):

    def __init__(self,config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4*config.n_embd)
        self.gelu = nn.GELU() # Gaussian Error Linear Units
        self.c_proj = nn.Linear(4*config.n_embd, config.n_embd)
    
    def forward(self,x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config) # Causal Self-Attention = Masked Self-Attention 
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = MLP(config) # MLP = Positionwise FFN
    
    def forward(self,x):
        x = x + self.attn(self.ln_1(x)) # Prenorm 
        x = x + self.mlp(self.ln_2(x)) 
        return x 

@dataclass 
class GPTConfig:
    block_size: int = 1024 # maximum number of positions = context window 
    vocab_size: int = 50257 # number of tokens (gpt2-tokenizer)
    n_embd: int = 768 # embedding dimension (tokens -> vectors)
    n_layer: int = 12 # number of transformer layers 
    n_head: int = 12 # number of parallel heads in attention layer 

class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config 
        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd), # token embedding
            wpe = nn.Embedding(config.block_size, config.n_embd), # position embedding
            h = nn.ModuleList(Block(config) for _ in range(config.n_layer)),
            ln_f = nn.LayerNorm(config.n_embd), 
        )
        )
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False) # last decision -> next token 
    
    def forward(self, idx, targets = None): 
        # idx = (B,T) ... B is batch_size, T is sequence length 
        B,T = idx.size()
        assert T <= self.config.block_size 
        pos = torch.arange(0, T, dtype = torch.long, device=idx.device)
        tok_emb = self.transformer.wte(idx) 
        pos_emb = self.transformer.wpe(pos)
        x = tok_emb + pos_emb 

        for block in self.transformer.h:
            x=block(x)
        
        x = self.transformer.ln_f(x)
        logits = self.lm_head(x) # FC 
        loss = None 
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1))
        
        return logits, loss
    
    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = GPTConfig(**config_args)
        model = GPT(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # ignore these, just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # same, just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        # basically the openai checkpoints use a "Conv1D" module, but we only want to use a vanilla Linear
        # this means that we have to transpose these weights when we import them
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model


In [19]:
class DataLoaderLite:
    def __init__(self, B, T):
        self.B = B
        self.T = T

        # at init load tokens from disk and store them in memory
        with open('goethe.txt', 'r') as f:
            text = f.read()
        enc = tiktoken.get_encoding('gpt2')
        tokens = enc.encode(text)
        self.tokens = torch.tensor(tokens)
        print(f"loaded {len(self.tokens)} tokens")
        print(f"1 epoch = {len(self.tokens) // (B * T)} batches")

        # state
        self.current_position = 0

    def next_batch(self):
        B, T = self.B, self.T
        buf = self.tokens[self.current_position : self.current_position+B*T+1]
        x = (buf[:-1]).view(B, T) # inputs
        y = (buf[1:]).view(B, T) # targets
        # advance the position in the tensor
        self.current_position += B * T
        # if loading the next batch would be out of bounds, reset
        if self.current_position + (B * T + 1) > len(self.tokens):
            self.current_position = 0
        return x, y


In [34]:
import tiktoken 
import time 
train_loader = DataLoaderLite(B=8, T=64)

device = "mps" if torch.backends.mps.is_available() else ("cuda" if torch.cuda.is_available() else "cpu")
num_rt_seqs = 1
max_len  = 100

model = GPT(GPTConfig)

#model = GPT.from_pretrained("gpt2")

model.to(device)
enc = tiktoken.get_encoding("gpt2")



optimizer = torch.optim.AdamW(model.parameters(), lr = 3e-4, betas = (0.9, 0.95), eps= 10e-8)
for i in range(1):
    t0=time.time()
    optimizer.zero_grad()
    x, y = train_loader.next_batch()
    x, y= x.to(device), y.to(device)
    with torch.autocast(device_type=device, dtype = torch.bfloat16): # 16 bit floating point
        logits,loss = model(x,y)
    loss.backward() 
    optimizer.step() 
    dt = time.time() - t0
    tokens_processed = train_loader.B * train_loader.T
    tokens_per_sec = tokens_processed / dt
    print(f"step {i}, loss: {loss.item()}, dt: {dt*1000:.2f} ms, tokens/sec: {tokens_per_sec:.2f}")

loaded 66404 tokens
1 epoch = 129 batches
step 0, loss: 11.0089111328125, dt: 181.85 ms, tokens/sec: 2815.43


In [33]:
model.eval()
model.to(device)
#tokens = enc.encode("You are helpful assistant. What is 2+2?")
tokens = enc.encode("FAUST") # Prompt
tokens = torch.tensor(tokens, dtype = torch.long)
tokens = tokens.unsqueeze(0).repeat(num_rt_seqs, 1)
x = tokens.to(device)

#torch.manual_seed(42)

while x.size(1) < max_len:
    with torch.no_grad():
        logits,loss = model(x)
        logits = logits[:,-1,:]
        probs = F.softmax(logits, dim = -1)
        top_probs, top_ind = torch.topk(probs, 50, dim = -1)
        ix = torch.multinomial(top_probs,1)
        xcol = torch.gather(top_ind, -1, ix)
        x = torch.cat((x,xcol), dim = 1)
#
for i in range(num_rt_seqs):
    tokens = x[i, :max_len].tolist()
    decoded = enc.decode(tokens)
    print(">", decoded)

> FAUST Here
L will do you, 
Be your heart, we of love her.							

MARGARET Than his hand,
That draws in the love inside,
And let him, there’s quite so without rest,
Only he has gone,
Your heart such a you, you,																
Then through be so
